In [1]:
from Paper_Results import TVSSegmentation #as pr
import Segmantation_lib as sg
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
trSegem=TVSSegmentation(tvs="g",season=1,episode=1)

You have chosen :
Tv-Series: Game of Thrones
Season:  1
Episode: 1
Subtile File :  /people/berhe/Bureau/TLP_thesis/subtitles/GoT/English/GameOfThrones.Season01.Episode01.en.srt


In [3]:
framesFeatures, timeStamp,shot=trSegem.load_Data()

In [4]:
manScene_boundry=trSegem.load_ManAnnotation()
_,manShot_boundry=sg.getShots(shot)
manClstr_ref,sceneMakar=trSegem.frameGroundClusters(manShot_boundry)
manScene_boundry=[i/1000 for i in manScene_boundry]

In [5]:
avgFeatures=trSegem.getAvgFeatures(framesFeatures,timeStamp,manShot_boundry)
randSampleFeatures,uniSmplFeatures=trSegem.getRandomFeat(framesFeatures,timeStamp,manShot_boundry)
#uniSmplFeatures=trSegem.getUniformSamples(framesFeatures,timeStamp,manShot_boundry)

avgFeatures=np.array(avgFeatures)
randSampleFeatures=np.array(randSampleFeatures)
randSampleFeatures=randSampleFeatures.reshape((randSampleFeatures.shape[0],randSampleFeatures.shape[1]*randSampleFeatures.shape[2]))
uniSmplFeatures=np.array(uniSmplFeatures)
uniSmplFeatures=uniSmplFeatures.reshape((uniSmplFeatures.shape[0],uniSmplFeatures.shape[1]*uniSmplFeatures.shape[2]))

In [6]:
uniSmplFeatures.shape,randSampleFeatures.shape

((980, 125440), (980, 125440))

In [8]:
framesFeatures.shape,len(manShot_boundry),randSampleFeatures.shape,uniSmplFeatures.shape

((88645, 25088), 980, (980, 125440), (980, 125440))

In [9]:
shot_TxtEmb=trSegem.getShotTextFeat(manShot_boundry)
shot_TxtEmb=np.array(shot_TxtEmb)
shot_TxtEmb.shape,avgFeatures.shape,randSampleFeatures.shape

conctAvgFeat=np.concatenate((avgFeatures,shot_TxtEmb),axis=1)
conctRandFeat=np.concatenate((randSampleFeatures,shot_TxtEmb),axis=1)
conctUniFeat=np.concatenate((uniSmplFeatures,shot_TxtEmb),axis=1)
conctAvgFeat.shape, conctRandFeat.shape

shottimenp=np.array(manShot_boundry[0:len(conctAvgFeat)])
conctAvgFeat_Time=np.column_stack((conctAvgFeat,shottimenp))
conctRandFeat_Time=np.column_stack((conctRandFeat,shottimenp))
conctUniFeat_Time=np.column_stack((conctUniFeat,shottimenp))
conctAvgFeat_Time.shape, conctRandFeat_Time.shape,shottimenp.shape

dist_AvgFeat=trSegem.getSimilarityMatrix(conctAvgFeat,'cosine')
dist_RandFeat=trSegem.getSimilarityMatrix(conctRandFeat,'cosine')
dist_UniFeat=trSegem.getSimilarityMatrix(conctUniFeat,'cosine')
dist_AvgFeat_Time=trSegem.getSimilarityMatrix(conctAvgFeat_Time,'cosine')
dist_RandFeat_Time=trSegem.getSimilarityMatrix(conctRandFeat_Time,'cosine') 
dist_UniFeat_Time=trSegem.getSimilarityMatrix(conctUniFeat_Time,'cosine')

In [10]:
#pr.saveClstrresults(dist_AvgFeat_Time,manClstr_ref)

In [30]:
df_tmp=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
df=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
df1=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
dfBest=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision','season','episode'))
clstAlgo=['KMeans','specteral','affinity']
kc=[[7,5,6,4],[6,4,3,3],[9,7,7,5]]
for i, clAl in enumerate(clstAlgo):
    avgFeatClusters=trSegem.getClusters(40,clAl,dist_AvgFeat)
    randFeatClusters=trSegem.getClusters(40,clAl,dist_RandFeat)
    uniFeatClusters=trSegem.getClusters(40,clAl,dist_UniFeat)
    avgFeatClusters=avgFeatClusters.tolist()
    randFeatClusters=randFeatClusters.tolist()
    uniFeatClusters=uniFeatClusters.tolist()
    avgFeatClusters_Time=trSegem.getClusters(40,clAl,dist_AvgFeat_Time)
    randFeatClusters_Time=trSegem.getClusters(40,clAl,dist_RandFeat_Time)
    uniFeatClusters_Time=trSegem.getClusters(40,clAl,dist_UniFeat_Time)
    avgFeatClusters_Time=avgFeatClusters_Time.tolist()
    randFeatClusters_Time=randFeatClusters_Time.tolist()
    uniFeatClusters_Time=uniFeatClusters_Time.tolist()

    sepPos=sg.sequenceSegmentation(avgFeatClusters,kc[i][0],kc[i][1])
    sepPos1=sg.sequenceSegmentation(randFeatClusters,kc[i][0],kc[i][1])
    sepPos2=sg.sequenceSegmentation(uniFeatClusters,kc[i][0],kc[i][1])
    sceneTime=[]
    sceneTime1=[]
    sceneTime2=[]
    sepPos_Time=sg.sequenceSegmentation(avgFeatClusters_Time,kc[i][2],kc[i][3])
    sepPos1_Time=sg.sequenceSegmentation(randFeatClusters_Time,kc[i][2],kc[i][3])
    sepPos2_Time=sg.sequenceSegmentation(uniFeatClusters_Time,kc[i][2],kc[i][3])
    sceneTime_Time=[]
    sceneTime1_Time=[]
    sceneTime2_Time=[]
    for i in sepPos:
        sceneTime.append(manShot_boundry[i])
    for i in sepPos1:
        sceneTime1.append(manShot_boundry[i])
    for i in sepPos2:
        sceneTime2.append(manShot_boundry[i])
    for i in sepPos2_Time:
        sceneTime2.append(manShot_boundry[i])
    for i in sepPos_Time:
        sceneTime.append(manShot_boundry[i])
    for i in sepPos1_Time:
        sceneTime1.append(manShot_boundry[i])
    print('Auto',clAl,':',len(sepPos),len(sepPos1),len(sepPos_Time),len(sepPos1_Time),'the Manual:',len(manScene_boundry))
    tMan,tMan_bin,tMan_bin_old,sceneTime_man_bin=trSegem.getTruthValue_Man(manShot_boundry,manScene_boundry)
    len(tMan),len(tMan_bin),len(tMan_bin_old),len(sceneTime_man_bin)

    tAuto,tAuto_bin,tAuto_bin_old,sceneTime_Auto_bin=trSegem.getTruthValues_Auto(sepPos,manShot_boundry)
    tAuto1,tAuto_bin1,tAuto_bin_old1,sceneTime_Auto_bin1=trSegem.getTruthValues_Auto(sepPos1,manShot_boundry)
    tAuto_Time,tAuto_bin_Time,tAuto_bin_old_Time,sceneTime_Auto_bin_Time=trSegem.getTruthValues_Auto(sepPos_Time,manShot_boundry)
    tAuto2,tAuto_bin2,tAuto_bin_old2,sceneTime_Auto_bin2=trSegem.getTruthValues_Auto(sepPos2,manShot_boundry)
    tAuto_Time2,tAuto_bin_Time2,tAuto_bin_old_Time2,sceneTime_Auto_bin_Time2=trSegem.getTruthValues_Auto(sepPos2_Time,manShot_boundry)
    tAuto1_Time,tAuto_bin1_Time,tAuto_bin_old1_Time,sceneTime_Auto_bin1_Time=trSegem.getTruthValues_Auto(sepPos1_Time,manShot_boundry)
    
    truth_valueList=[tAuto,tAuto_bin,tAuto_bin_old,tAuto1,tAuto_bin1,tAuto_bin_old1,tAuto2,tAuto_bin2,tAuto_bin_old2]
    truth_valueList_Time=[tAuto_Time,tAuto_bin_Time,tAuto_bin_old_Time,tAuto1_Time,tAuto_bin1_Time,tAuto_bin_old1_Time,tAuto_Time2,tAuto_bin_old_Time2,tAuto_bin_old_Time2]
    truth_valueListMan=[tMan,tMan_bin,tMan_bin_old,tMan,tMan_bin,tMan_bin_old,tMan,tMan_bin,tMan_bin_old]
    
    for i in range(len(truth_valueList)):
        if i%3==0:
            results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList[i],rType='macro')
        else:
            results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList[i],rType='binary')
        row=pd.Series({'winDiff':results[0], 'pk':results[1],'coverage':results[2],'purity':results[3],'recall':results[4],'precision':results[5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
        df = df.append(row)
        df_tmp=df_tmp.append(row)
        
    row1=pd.Series({'winDiff':df_tmp.iloc[1][0], 'pk':df_tmp.iloc[1][1],'coverage':df_tmp.iloc[0][2],'purity':df_tmp.iloc[0][3],'recall':df_tmp.iloc[1][4],'precision':df_tmp.iloc[1][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
    dfBest=dfBest.append(row1)
    row1=pd.Series({'winDiff':df_tmp.iloc[4][0], 'pk':df_tmp.iloc[4][1],'coverage':df_tmp.iloc[3][2],'purity':df_tmp.iloc[3][3],'recall':df_tmp.iloc[4][4],'precision':df_tmp.iloc[4][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
    dfBest=dfBest.append(row1)
    row1=pd.Series({'winDiff':df_tmp.iloc[7][0], 'pk':df_tmp.iloc[7][1],'coverage':df_tmp.iloc[6][2],'purity':df_tmp.iloc[6][3],'recall':df_tmp.iloc[7][4],'precision':df_tmp.iloc[7][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
    dfBest=dfBest.append(row1)
    df_tmp=df_tmp.drop([clAl])
    
    for i in range(len(truth_valueList_Time)):
        if i%3==0:
            results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList_Time[i],rType='macro')
        else:
            results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList_Time[i],rType='binary')
        row=pd.Series({'winDiff':results[0], 'pk':results[1],'coverage':results[2],'purity':results[3],'recall':results[4],'precision':results[5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
        df1 = df1.append(row)
        df_tmp=df_tmp.append(row)
        
    row1=pd.Series({'winDiff':df_tmp.iloc[1][0], 'pk':df_tmp.iloc[1][1],'coverage':df_tmp.iloc[0][2],'purity':df_tmp.iloc[0][3],'recall':df_tmp.iloc[1][4],'precision':df_tmp.iloc[1][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
    dfBest=dfBest.append(row1)
    row1=pd.Series({'winDiff':df_tmp.iloc[4][0], 'pk':df_tmp.iloc[4][1],'coverage':df_tmp.iloc[3][2],'purity':df_tmp.iloc[3][3],'recall':df_tmp.iloc[4][4],'precision':df_tmp.iloc[4][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
    dfBest=dfBest.append(row1)
    row1=pd.Series({'winDiff':df_tmp.iloc[7][0], 'pk':df_tmp.iloc[7][1],'coverage':df_tmp.iloc[6][2],'purity':df_tmp.iloc[6][3],'recall':df_tmp.iloc[7][4],'precision':df_tmp.iloc[7][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
    dfBest=dfBest.append(row1)
    df_tmp=df_tmp.drop([clAl])

Auto KMeans : 44 43 43 34 the Manual: 44
Auto specteral : 45 42 36 64 the Manual: 44
Auto affinity : 36 40 44 46 the Manual: 44


In [31]:
df.iloc[0][2],df.iloc[0][3],df.iloc[3][2],df.iloc[3][3]

(0.759, 0.814, 0.776, 0.839)

In [32]:
df1

,winDiff,pk,coverage,purity,recall,precision,episode,season
KMeans,1.660,0.058,0.748,0.744,0.086,0.063,1,1
KMeans,0.151,0.086,0.956,0.956,0.955,0.955,1,1
KMeans,0.542,0.411,0.956,0.956,0.023,0.023,1,1
KMeans,1.362,0.083,0.844,0.788,0.023,0.003,1,1
KMeans,0.112,0.064,0.965,0.956,0.971,0.962,1,1
KMeans,0.397,0.297,0.965,0.956,0.163,0.206,1,1
KMeans,1.554,0.068,0.757,0.844,0.122,0.115,1,1
KMeans,1.822,0.923,0.955,0.956,0.041,0.864,1,1
KMeans,0.451,0.344,0.955,0.956,0.140,0.136,1,1
specteral,0.112,0.007,0.858,0.451,0.072,0.057,1,1


In [33]:
df.iloc[0][2],df.iloc[0][3],df.iloc[3][2],df.iloc[3][3],df1.iloc[0][2],df1.iloc[0][3],df1.iloc[3][2],df1.iloc[3][3]

(0.759, 0.814, 0.776, 0.839, 0.748, 0.744, 0.844, 0.788)

In [34]:
dfBest

,winDiff,pk,coverage,purity,recall,precision,season,episode
KMeans,0.157,0.087,0.759,0.814,0.954,0.955,1,1
KMeans,0.145,0.080,0.776,0.839,0.958,0.958,1,1
KMeans,0.145,0.083,0.713,0.852,0.953,0.960,1,1
KMeans,0.151,0.086,0.748,0.744,0.955,0.955,1,1
KMeans,0.112,0.064,0.844,0.788,0.971,0.962,1,1
KMeans,1.822,0.923,0.757,0.844,0.041,0.864,1,1
specteral,0.143,0.078,0.666,0.788,0.958,0.960,1,1
specteral,0.133,0.074,0.738,0.795,0.962,0.961,1,1
specteral,0.143,0.079,0.742,0.793,0.957,0.960,1,1
specteral,0.139,0.074,0.858,0.451,0.965,0.958,1,1


In [ ]:
df.to_csv('ResultsPaper/GoT/Results_segmentaion_TextEmbe_Augemnted_'+trSegem.Season+'.'+trSegem.Episode+'.csv')
df1.to_csv('ResultsPaper/GoT/Results_segmentaion_TimeAugmented_'+trSegem.Season+'.'+trSegem.Episode+'.csv')

In [23]:
df_tmp.shape

(0, 8)